In [3]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm
# Hello

### Chrome Driver

In [4]:
# Before starting scarping, we should download "Chromedriver"

# Path to chrome driver
ChromeDriver = "/Users/huanglinchun/Desktop/VS Code Files/chromedriver_mac64/chromedriver.exe"

# launch the driver (it will create an empty google chrome page)
driver = webdriver.Chrome(ChromeDriver)


/var/folders/gl/stf2yn_54sdd_k_jlfmc4nvr0000gn/T/ipykernel_40576/1725469750.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriver)


### User Imput

In [13]:
# get user input for routes
'''
User Input Order:
1. 1st airline departure
2. 1st airline destination
3. 2nd airline departure
4. 2nd airline destination
.
.
In this case, we have two combinations:
[NRT TPE][TPE NRT][HND TPE][TPE HND]
'''
sources = []
destinations = []
print("Please enter -1 when done.")
print("-"*10)
while True:
    sources.append(input("From which city?\n"))
    if "-1" in sources: 
        sources.pop(-1)
        break
    destinations.append(input("Where to?\n"))
    if "-1" in destinations: 
        sources.pop(-1)
        destinations.pop(-1)
        break
    print("-"*10)

print("\nRoutes:")
for i in range(len(sources)):
    print(f"{sources[i]} => {destinations[i]}")

Please enter -1 when done.
----------
----------

Routes:
TPE => HND


In [7]:
# get user input for period (start and end date)
start_date = np.datetime64(input('Start Date, Please use YYYY-MM-DD format only '))
end_date = np.datetime64(input('End Date, Please use YYYY-MM-DD format only '))
days = end_date - start_date
num_days = days.item().days

'''
np.datetime64 object can simply use + - to control date! very smart
'''

### Define Functions for Scraping

In [39]:
def get_airlines(soup):
    # 1. Create an empty list for storing airline data
    airline = []

    # 2. Use "find_all" to find all 'div' with class = VY2U in the page(soup)
    #    type(airlines) = list
    airlines = soup.find_all('div', class_ = "VY2U")     # Result Set     
    for i in airlines:
        # 3. Use for loop to find the text with airline data in each item in "airlines"
        #    [HTML OBJECT].find("{HTML ATTRIBUTE}", class_="", ...)
        a = i.find(class_= 'c_cgF')                      # Tag

        #    [HTML OBJECT].text => Return all texts in the object
        airline.append(a.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('div',class_='vmXl-mod-variant-default')

    for i in stops:
        for j in i.find_all('span',class_='JWEO-stops-text'):
            stops_list.append(j.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='f8F1-price-text-container')

    for i in price:
        for j in i.find_all('div', class_='f8F1-price-text'):
            prices.append(j.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='xdW8')
    for i in duration:
        for j in i.find_all('div',class_='vmXl'):
            duration_list.append(j.text)
    return duration_list

### Scraping Loops

In [ ]:
from selenium.webdriver.common.by import By

# 4 sources so we have 4 destinations, then this loop will run 4 times.
for i in range(len(sources)):
    column_names = ["Airline", "Source", "Destination","Duration" ,"Total stops", "Price","Date"]
    df = pd.DataFrame(columns = column_names)

    # We set the date from 2022-05-01 to 2023-05-01, so there will be about 366 days
    # This loop will run for 366 times
    for j in tqdm(range(num_days+1)):
        
        # close and open driver every 10 days to avoid captcha
        driver = webdriver.Chrome(ChromeDriver)
        
        if j % 10 == 0:
            driver.quit()
            driver = webdriver.Chrome(ChromeDriver)
            #, chrome_options=chromeOptions)
        
        
        # Observing the structures of URL is also an useful approach
        url = f"https://www.kayak.com/flights/{sources[i]}-{destinations[i]}/{start_date+j}"
        driver.get(url)
        sleep(8)
        
        # Oneway = driver.find_elements(By.XPATH, "//*[contains(text(), 'One-way')]")
        
        # click show more button to get all flights
        
        # find_element is a function of "WebDriver", not BS4
        try:
            show_more_button = driver.find_element(By.CLASS_NAME, "ULvh-button")
        except:
            pass

        while True:
            try:
                show_more_button.click()
                driver.implicitly_wait(10)
            except:
                break
        
        # Use "BeautifulSoup" to get web page parser
        # BeautifulSoup({WEBDRIVER}.page_source, 'html.parser')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airlines(soup)             # Return a list of airline data
        total_stops = get_total_stops(soup)       # Return a list of total stop data
        prices = get_price(soup)                  # Return a list of price data
        duration = get_duration(soup)             # Return a list of duration data
        
        # Use df.append({COLUMN: ARRAYS, ...})
        df = df.append(pd.DataFrame({
            'Airline': airlines,
            'Duration': duration,
            'Total stops' : total_stops,
            'Price' : prices,
            'Date' : start_date+j
                                    }))
    
    # Because all of the sources and destinations in this dataframe are the same
    df['Source'] = sources[i]
    df['Destination'] = destinations[i]
    df = df.replace('\n','', regex=True)
    df = df.reset_index(drop = True)
    
    # save data as csv file for each route
    df.to_csv(f'{sources[i]}_{destinations[i]}.csv',index=False)
    print(f"Succesfully saved {sources[i]} => {destinations[i]} route as {sources[i]}_{destinations[i]}.csv ")
    
driver.quit()

### Scraping Testing Codes

In [41]:
driver = webdriver.Chrome(ChromeDriver)
url = "https://www.kayak.com/flights/TPE-NRT/2023-05-12"
driver.get(url)
sleep(5)

soup = BeautifulSoup(driver.page_source, 'html.parser')
price_1 = get_airlines(soup)
print(price_1)




/var/folders/gl/stf2yn_54sdd_k_jlfmc4nvr0000gn/T/ipykernel_40576/1774845910.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriver)


['Jetstar Japan', 'Peach', 'Tigerair Taiwan', 'VietJet Air', 'Tigerair Taiwan', 'VietJet Air', 'VietJet Air', 'China Airlines', 'Scoot', 'Jeju Air', 'Asiana Airlines', 'Asiana Airlines', 'Asiana Airlines', 'Asiana Airlines', 'Hahn Air Systems']
